### Narrative Explanation:

1. **Contributor Fund:**  
   - CSL forces corporate contributions and structured incentives, growing the fund sustainably over time.  
   - MIT allows unrestricted adoption but relies on voluntary contributions, leading to slow fund growth.

2. **Corporate Contributions:**  
   - CSL line shows substantial contributions from large organizations due to license requirements.  
   - MIT shows near-zero contributions because corporations can take without giving back.

3. **Active Developers:**  
   - CSL increases active developers through incentives and fund-supported payouts.  
   - MIT grows slower as there is less structured support for developers.

4. **Total Adoption:**  
   - MIT initially outpaces CSL because of unrestricted corporate adoption.  
   - CSL adoption grows more slowly, but the ecosystem is healthier: more developers supported, more sustainable fund growth.

**Conclusion:**  
This visualization clarifies that although MIT adoption grows quickly, CSL creates a **stronger, self-sustaining community** and a larger contributor fund, aligning incentives between corporations and open-source developers.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
np.random.seed(42)

# --- Agents ---
agents = {
    'Individuals': {'count':500, 'base_revenue':1.0, 'base_adopt_prob':0.01},
    'SmallOrgs': {'count':50, 'base_revenue':50.0, 'base_adopt_prob':0.05},
    'LargeOrgs': {'count':5, 'base_revenue':2000.0, 'base_adopt_prob':0.2},
}

T = 120

# --- Simulation ---
def simulate(params, T_steps=None, seed=42):
    if T_steps is None: T_steps=T
    np.random.seed(seed)
    
    contributor_fund = 1000.0
    active_devs = 50.0
    
    metrics = {
        't':[], 'contributor_fund':[], 'corporate_contribution':[], 'market_concentration':[],
        'total_adoption':[], 'total_revenue':[], 'contributors_paid':[],
        'forged_forks':[], 'active_developers':[]
    }
    
    for t in range(T_steps):
        adoption = 0.0
        revenue = 0.0
        hhi_components = []
        effort = 0.0
        forks_this_step = 0
        corporate_contribution = 0.0
        
        for ag_name, ag in agents.items():
            count, base_rev, adopt_prob = ag['count'], ag['base_revenue'], ag['base_adopt_prob']
            
            if ag_name == 'LargeOrgs':
                # Adoption reduced by barriers in CSL
                adopt_prob *= params.get('large_adopt_multiplier', 1.0)
                adopt_prob *= (1 - params.get('barrier_to_entry', 0.0))
                forks = np.random.binomial(count, params.get('fork_penalty', 0)*0.5)
                adopters = np.random.binomial(count-forks, min(0.999, adopt_prob))
                forks_this_step += forks
                
                # CSL forces large orgs to contribute
                contribution = adopters * base_rev * params.get('corporate_contribution_pct',0.0)
                corporate_contribution += contribution
                contributor_fund += contribution
                
            else:
                adopters = np.random.binomial(count, min(0.999, adopt_prob + params.get('network_effect',0.0)*(active_devs/100.0)))
            
            rev = adopters*base_rev
            revenue += rev
            effort += adopters*params.get('contributor_incentive',0.1)
            adoption += adopters
            hhi_components.append(rev**2)
        
        # Enforcement & payout
        enforcement_cost = contributor_fund*params.get('enforcement_cost_pct',0.0)
        contributor_fund -= enforcement_cost
        payout = min(contributor_fund*0.02, effort*0.05)
        contributor_fund -= payout
        
        # Active developers grow with adoption & payout
        active_devs += 0.1*(adoption/10.0) + 0.2*(payout/100.0) - 0.05*(enforcement_cost/100.0)
        active_devs = max(1.0, active_devs)
        
        # Record metrics
        metrics['t'].append(t)
        metrics['contributor_fund'].append(contributor_fund)
        metrics['corporate_contribution'].append(corporate_contribution)
        metrics['market_concentration'].append(sum(hhi_components)/(revenue**2) if revenue>0 else 0)
        metrics['total_adoption'].append(adoption)
        metrics['total_revenue'].append(revenue)
        metrics['contributors_paid'].append(payout)
        metrics['forged_forks'].append(forks_this_step)
        metrics['active_developers'].append(active_devs)
    
    return pd.DataFrame(metrics)

# --- Baseline Scenarios ---
base_params = {
    'MIT': {'large_adopt_multiplier':1.0,'contributor_share_pct':0.01,'corporate_contribution_pct':0.0,
            'barrier_to_entry':0.0,'contributor_incentive':0.1,'enforcement_cost_pct':0.0,'fork_penalty':0.0,'network_effect':0.001},
    'CSL': {'large_adopt_multiplier':0.3,'contributor_share_pct':0.10,'corporate_contribution_pct':0.2,
            'barrier_to_entry':0.3,'contributor_incentive':0.3,'enforcement_cost_pct':0.02,'fork_penalty':0.6,'network_effect':0.005}
}

res_mit = simulate(base_params['MIT'])
res_csl = simulate(base_params['CSL'])

# --- Plotting in clean subplots ---
fig, axs = plt.subplots(2,2, figsize=(14,10), sharex=True)
colors = {'MIT':'#1f77b4', 'CSL':'#ff7f0e'}

# Contributor fund
axs[0,0].plot(res_mit['t'], res_mit['contributor_fund'], label='MIT', color=colors['MIT'])
axs[0,0].plot(res_csl['t'], res_csl['contributor_fund'], label='CSL', color=colors['CSL'])
axs[0,0].set_title('Contributor Fund')
axs[0,0].legend(); axs[0,0].grid(True)

# Corporate contributions
axs[0,1].plot(res_mit['t'], res_mit['corporate_contribution'], label='MIT', color=colors['MIT'])
axs[0,1].plot(res_csl['t'], res_csl['corporate_contribution'], label='CSL', color=colors['CSL'])
axs[0,1].set_title('Corporate Contributions to Fund')
axs[0,1].legend(); axs[0,1].grid(True)

# Active developers
axs[1,0].plot(res_mit['t'], res_mit['active_developers'], label='MIT', color=colors['MIT'])
axs[1,0].plot(res_csl['t'], res_csl['active_developers'], label='CSL', color=colors['CSL'])
axs[1,0].set_title('Active Developers')
axs[1,0].legend(); axs[1,0].grid(True)

# Total adoption
axs[1,1].plot(res_mit['t'], res_mit['total_adoption'], label='MIT', color=colors['MIT'])
axs[1,1].plot(res_csl['t'], res_csl['total_adoption'], label='CSL', color=colors['CSL'])
axs[1,1].set_title('Total Adoption (All Agents)')
axs[1,1].legend(); axs[1,1].grid(True)

for ax in axs[1,:]:
    ax.set_xlabel('Time Steps (Months)')

plt.suptitle('MIT vs CSL Simulation of License & Community Dynamics', fontsize=16)
plt.tight_layout(rect=[0,0,1,0.96])
plt.show()